# Notebook para el cálculo de la frontera eficiente y simulación de carteras

Archivo que toma como entrada un archivo de Excel con el formato habitual y que permite:
- Análisis de la frontera eficiente de los activos del archivo
- Creación de carteras aleatorias (incluyendo carteras 'extremas')
- Filtrado de las carteras 
- Creación de gráficos en dónde se representa toda la información de forma interactiva

### Instrucciones de uso:
- Ejecute cada celda una detrás de otra y vaya interactuando con los botones y cuadros de texto que aparezcan. Si no se hace esto, el archivo nos dará un mensaje de error y habrá que ejecutar de nuevo todas las celdas accediendo previamente a las siguientes opciones: 'Kernel' > 'Restart'. Esto reseteará el programa interno y permitirá ejecutar de nuevo las celdas. El usuario más experimentado con el manejo del presente Notebook podrá ejecutar las celdas anteriores al error para solventarlo sin necesidad de reestablecer el 'kernel'. Si en cualquier momento de la ejecución se muestra algún mensaje de error, el procedimiento que se ha descrito será el habitual para resolver cualquier tipo de error. 
- La primera celda del 'bloque de celdas de configuración' contiene las librerías que habrá que instalar antes de la primera ejecución del Notebook. La instalación de la librería 'PyPortfolioOpt' requiere de instrucciones específicas de instalación que se detallarán a continuación. Una vez que se hayan instalado ambas librerías mediante la ejecución de esta celda, ya no habrá que ejecutarla nunca más. 
- Al ejecutar la celda número 2 con nombre 'Celda 2: generación de carteras', el programa dará al usuario la opción de pulsar dos botones, 'Generar carteras' y ' Guardar carteras'. Es obligatorio pulsar en el primero de ellos pero opcional en el segundo. Deberán pulsarse en ese orden, si no, el programa dará error, ya que tratará de guardar unas carteras que no se han generado todavía. 
- En la celda número 3 con nombre 'Celda 3: filtrado de carteras', será necesario pulsar al menos una vez el botón de 'Filtrar carteras'.
- En la celda número 5 con nombre 'Celda 5: gráficas', aparecerán tres botones tras la primera ejecución, cada vez que se ejecute este archivo: 'Añadir', 'Visualizar', 'Borrar' y 'Poner a cero'; además de unos cuadros de texto, tantos como el número de activos que se estén considerando. Para visualizar los datos generados directamente, pulsar el botón 'Borrar'. Estos botones y los cuadros de texto, permiten añadir carteras a petición expresa del usuario al gráfico final. El botón 'Poner a cero' pondrá los pesos de activo a cero, que por defecto aparecerán distribuídos equitativamente en cada activo. El botón 'Añadir', añadirá la cartera que se haya introducido al gráfico y a una lista dónde se guardarán todas las carteras personalizadas que se añadan. El botón 'Visualizar' pintará la cartera en el gráfico pero no la guardará. Y finalmente, el botón 'Borrar' reseteará a cero la lista de carteras que ha introducido manualmente el usuario. 

##### Instrucciones de instalación de la librería 'PyPortfolioOpt' (sacado del sitio web oficial)
Sitio web oficial de <a href="https://pyportfolioopt.readthedocs.io/en/latest/">PyPortfolioOpt</a>. Para una instalación en un sistema operativo distinto de Windows consultar en la página web.

- Antes de instalar PyPortfolioOpt, habrá que instalar C++ en el ordenador desde <a href="https://visualstudio.microsoft.com/thank-you-downloading-visual-studio/?sku=BuildTools&rel=16">aquí</a>. En <a href="https://drive.google.com/file/d/0B4GsMXCRaSSIOWpYQkstajlYZ0tPVkNQSElmTWh1dXFaYkJr/view">este</a> sitio hay instrucciones habituales que el usuario deberá consultar para la instalación. 
- Una vez instalado, se puede proceder a la ejecución de la primera celda ejecutable de este Notebook, o podrá instalar las librerías necesarias desde la consola de Anaconda, 'Anaconda Prompt (anaconda3)'.

#### Bloque de celdas de configuración

In [1]:
!conda install pip -y # Gestor de liberías
!pip install PyPortfolioOpt # Libería para el cálculo de la frontera eficiente
!pip install ipywidgets # Librería para la interacción con el usuario: botones, cuadros de texto...

###### Ejecutar a partir de aquí una vez que ya estén instaladas todas las librerías necesarias

In [1]:
import pandas as pd
import pypfopt as pfo
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import ipywidgets as widgets
import matplotlib.ticker as mtick
import copy, datetime, itertools, glob


from pypfopt import plotting
from PyQt5.QtWidgets import QFileDialog
from openpyxl import load_workbook
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from cycler import cycler

pd.options.display.float_format = '{:.2%}'.format

In [2]:
%gui qt

from PyQt5.QtWidgets import QFileDialog

def gui_fname():
    dir ='./'
    fname = QFileDialog.getOpenFileName(None, "Seleccione archivo de datos", 
                dir, filter="All files (*);; SM Files (*.sm)")
    return fname[0]   

#### Celda 1: elección del archivo

In [3]:
excel = gui_fname()

datos_esperados = pd.read_excel(excel, sheet_name="Datos esperados")
datos_esperados = datos_esperados.rename({datos_esperados.columns[0]:'Datos'}, axis=1).set_index('Datos')

mat_cov = pd.read_excel(excel, sheet_name="Matriz de covarianzas")
mat_cov = mat_cov.rename({mat_cov.columns[0]:'Mat. cov.'}, axis=1).set_index('Mat. cov.')

# Chequear tipo de dato
rentabilidades = datos_esperados.iloc[0, :].tolist()

if isinstance(rentabilidades[0], str):
    rentabilidades = [10**(-2)*float(x[:-1]) for x in rentabilidades]
else:
    rentabilidades = [float(x) for x in rentabilidades]
    
mat_covarianzas = mat_cov.to_numpy()

# Chequear 'Histórico' o 'Esperado' y cambiar matriz de covarianzas
clase = pd.read_excel(excel, sheet_name="Datos esperados", usecols="A").columns[0]

if clase == "Esperado": # Modificar matriz de covarianzas
    np.fill_diagonal(mat_covarianzas, (datos_esperados.iloc[1, :])**2/365)

# Funciones de utilidad
def vol_cartera(pesos, mat_cov):
    return np.sqrt(365*(np.array(pesos) @ np.array(mat_cov) @ np.array(pesos)))

def rent_cartera(pesos, rent_anu):
    ddf = np.array(pesos).dot(np.array(rent_anu))
    return ddf

def sharpe(rent, vol):
    return rent/vol

nombres = datos_esperados.columns.tolist()

#### Celda 2: generación de carteras

In [4]:
num_carteras = ['joy']
style = {'description_width': 'initial'}    

def get_step(num):
    steps = [0.005, 0.02, 0.05, 0.05, 0.1, 0.1, 0.1, 0.1, 0.125, 0.125, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]
    
    if num < 3:
        raise Exception("El número de activos debe ser mayor de 2")
    
    if num <= 20:
        return steps[num-3]
    else: return 0.5
    

def return_str_vars_anteriores(t_variables, indice):
    if indice == 0:
        return '0'
    else:
        return '+'.join(t_variables[:indice])
    

def get_pesos_distr():
    n_activos = len(nombres)
    st = get_step(n_activos)

    variables = ['x'+str(i+1) for i in range(n_activos-1)]

    instruccion = 'pd.DataFrame([('
    instruccion += ', '.join(variables)+', '
    instruccion += '1-('+('+').join(variables)+')) '
    instruccion += ' '.join(['for '+var+' in np.arange(0, 1-('+ return_str_vars_anteriores(variables, variables.index(var)) +')+'+str(st)+', '+str(st)+')' for var in variables])
    instruccion += '], columns=[*nombres])'

    t_pesos = eval(instruccion).to_numpy()
    
    return t_pesos
    

def on_button_clicked(x):    
    global pesos
    
    btn_guardar.button_style="success"
    
    try:
        num_carteras[0] = txt.value
    except:
        num_carteras[0] = 10000
    
    # Generamos pesos
    try:
        no = num_carteras[0]  
    except:
        no = 10000
        print("Error en asignación de número de carteras aleatorias")
    
    pesos_distr = get_pesos_distr()
    pesos = copy.deepcopy(pesos_distr.tolist())
    a_pesos = np.random.dirichlet(np.ones(len(nombres))-0.4, no)
    if len(a_pesos<0):
        [pesos.append(x) for x in a_pesos]
    
    # Mandamos un mensaje de confirmación
    with out:
        out.clear_output()
        display("%d carteras generadas - %s" % (len(pesos), str(datetime.datetime.now().strftime("%H:%M:%S"))))


def on_button_guardar_click(x):
    rent_simcarteras = []
    vols_simcarteras = []
    sr_simcarteras = []
    iden = []
    
    pesos_to_save = []

    for i, w in enumerate(pesos):
        rent_simcarteras.append(rent_cartera(w, rentabilidades))
        vols_simcarteras.append(vol_cartera(w, mat_covarianzas))
        sr_simcarteras.append(sharpe(rent_simcarteras[-1], vols_simcarteras[-1]))
        iden.append(str("Cartera "+str(i+1)))
    
    for i in range(len(pesos)):
        pesos_to_save.append(list(pesos[i])+[rent_simcarteras[i], vols_simcarteras[i], sr_simcarteras[i]])


    df = pd.DataFrame(pesos_to_save, columns=[*nombres, 'Rentabilidad', 'Volatilidad', 'Ratio de Sharpe'])
    try:
        df.to_excel("Output de carteras generadas.xlsx", sheet_name="Simulación de carteras")
        with out_guardar:
            out_guardar.clear_output()
            display("Datos guardados")
    except:
        with out_guardar:
            out_guardar.clear_output()
            display("No se ha podido guardar los datos")

txt = widgets.BoundedIntText(value=10000, 
                             min=1, 
                             max=10000000, 
                             description="Número de carteras aleatorias extras:", 
                             layout=widgets.Layout(position="center", margin='15px 0px 0px 20px'), 
                             style=style)

btn = widgets.Button(description="Generar carteras", layout=widgets.Layout(margin='30px 0px 0px 20px'), button_style="success")    
btn_guardar = widgets.Button(description="Guardar carteras", layout=widgets.Layout(margin="10px 0 0 20px"))
out = widgets.Output(layout=widgets.Layout(margin="30px 0% 0% 20px"))
out_guardar = widgets.Output(layout=widgets.Layout(margin="10px 0 0 20px"))

btn.on_click(on_button_clicked)
btn_guardar.on_click(on_button_guardar_click)

widgets.VBox([txt, widgets.HBox([btn, out]), widgets.HBox([btn_guardar, out_guardar])])


#### Celda 3: filtrado de carteras

In [5]:
limis = {}
num_carteras = ['t']

def click_filtrar(x):
    global pesos_filtrados, rent_simcarteras, vols_simcarteras, sr_simcarteras
    pesos_filtrados = []
    
    for n in nombres:
        limis[n] = (np.array(displays[n].value)/100).tolist()
        
    for f_p in pesos:
        control = True

        for i_n, n in enumerate(nombres):
            if limis[n][0] > f_p[i_n]:
                control = False

            elif limis[n][1] < f_p[i_n]:
                control = False

        if control:
            pesos_filtrados.append(f_p)
        
    rent_simcarteras = []
    vols_simcarteras = []
    sr_simcarteras = []

    for w in pesos_filtrados:
        rent_simcarteras.append(rent_cartera(w, rentabilidades))
        vols_simcarteras.append(vol_cartera(w, mat_covarianzas))
        sr_simcarteras.append(sharpe(rent_simcarteras[-1], vols_simcarteras[-1]))
        
    with out_filtrar:
        out_filtrar.clear_output()
        display("%d carteras restantes de %d" % (len(pesos_filtrados), len(pesos)))

for e in nombres:
    limis[e] = [0, 100]

    
displays = {}

for n in nombres:
    displays[n] = widgets.IntRangeSlider(min=0, max=100, value=[0, 100])

btn_filtrar = widgets.Button(description="Filtrar carteras", layout=widgets.Layout(margin='30px 0px 0px 0px'), button_style="success",) 
out_filtrar = widgets.Output(layout=widgets.Layout(margin="30px 0px 0px 10px"))

btn_filtrar.on_click(click_filtrar)

boxes = [widgets.HBox([widgets.Label(n), displays[n]], layout=widgets.Layout(justify_content="center")) for n in nombres]

widgets.Box(boxes+[btn_filtrar, out_filtrar], layout=widgets.Layout(flex_flow="column"))

Box(children=(HBox(children=(Label(value='Mercado monetario'), IntRangeSlider(value=(0, 100))), layout=Layout(…

#### Celda 4: cálculo de la frontera eficiente

In [6]:
def get_data_ef(ef, risk_range):
    mus, sigmas, ws = [], [], []
    
    # Creamos una cartera para cada valor del riesgo
    for param in risk_range:
        ef_i = copy.deepcopy(ef)
        
        try:
            ef_i.efficient_risk(param)
        except pfo.exceptions.OptimizationError:
            continue
    
        ret, sigma, _ = ef_i.portfolio_performance()
        w = []
        for i in range(len(ef_i.clean_weights())):
            w.append(ef_i.clean_weights()[i])
        
        mus.append(ret)
        sigmas.append(sigma)
        ws.append(w)
    
    return sigmas, mus, ws
    
    
    
ef = pfo.efficient_frontier.EfficientFrontier(rentabilidades, 365*mat_covarianzas)
risk_range = np.linspace(0.005, np.max(np.sqrt(365*np.diag(mat_covarianzas))), 1000)
    
vols, rets, w_ef = get_data_ef(ef, risk_range)

data_ef = []

for i in range(len(vols)):
    fila = []
    for j in range(len(w_ef[i])):
        fila.append(w_ef[i][j])
    fila.append(rets[i])
    fila.append(vols[i])
    fila.append(rets[i]/vols[i])
    
    data_ef.append(fila)

df_data_ef = pd.DataFrame(data_ef, columns=[*nombres, "Rentabilidad", "Volatilidad", "Ratio de Sharpe"])


# Guardamos en el excel pero en una pestaña diferente

if 'Output de carteras generadas.xlsx' in glob.glob('*'):
    book = load_workbook("Output de carteras generadas.xlsx")
    writer = pd.ExcelWriter("Output de carteras generadas.xlsx", engine="openpyxl")
    writer.book = book

    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

    df_data_ef.to_excel(writer, sheet_name="Frontera eficiente")

    writer.save()
else:
    df_data_ef.to_excel("Output de carteras generadas.xlsx", sheet_name="Frontera eficiente")

C:\Users\rafael.suarez\Anaconda3\lib\site-packages\pypfopt\efficient_frontier\efficient_frontier.py:336: RuntimeWarning: invalid value encountered in sqrt
  global_min_volatility = np.sqrt(1 / np.sum(np.linalg.inv(self.cov_matrix)))


#### Celda 5: gráficas

In [8]:
%matplotlib inline
# %matplotlib notebook

# Configuración matplotlib
colors = cycler(color=['#424242', '#9a999e', '#17375e', '#446cb3', '#94afe1', '#dce6f8', '#ca5d0c', '#caa60c', '#feb55b', '#ffd9aa', '#0F864A', '#77933C', '#BFBF16', '#341C83', '#AE9EE1', '#FEFE5B', '#C00000', '#ca870c',  '#caa60c'])

plt.ioff()
plt.rc('axes', prop_cycle=colors)

# Funciones
def draw():
    plt.figure(figsize=[15, 8])
    ax = plt.gca()

    # Plot ef
    plt.plot(vols, rets, label="Frontera eficiente", c="#538DD5", linewidth='5')
    # Plot carteras
    plt.scatter(vols_simcarteras, 
                rent_simcarteras, 
                alpha=0.7, 
                marker="+", 
                cmap="viridis_r",
                label="Simulación de carteras",
                c=sr_simcarteras)
    
    plt.colorbar(label="Ratio de Sharpe")
    
    for i, n in enumerate([n[:21] for n in nombres]):
        plt.scatter(np.sqrt(365*np.diag(mat_covarianzas))[i], rentabilidades[i], label=n, marker="^", s=100)

    # Plot carteras correctamente diversificadas
    plt.title("Frontera eficiente", fontsize=15, pad=30)
    plt.xlabel("Volatilidad", fontsize=12)
    plt.ylabel("Rentabilidad", fontsize=12)
    
    ax.set_xticks(np.arange(0, np.max(np.sqrt(365*np.diag(mat_covarianzas)))+0.02, step=0.02))
    ax.xaxis.set_major_formatter(mtick.FuncFormatter(lambda x, pos: str(str(int(np.round(x*100)))+'%')))
    ax.yaxis.set_major_formatter(mtick.FuncFormatter(lambda x, pos: str(str(int(np.round(x*100)))+'%')))

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    
    plt.legend(bbox_to_anchor=(1.15,1), loc="upper left")

    
def print_info():
    with out3:
        out3.clear_output()
        if len(info_extras) != 0:    
            for i in range(len(info_extras)):
                informacion = str(i+1)+": "
                
                for i_n, n in enumerate(nombres):
                    informacion += "{:.2%}".format(info_extras[i][i_n]) + " " + n + " | "
                
                
                display(informacion[:-3])

extras = []
info_extras = []


# Definiciones
intro_pesos_d = {}

btn_anadir = widgets.Button(description="Añadir", layout=widgets.Layout(margin="10px 0px 15px 20px"), button_style="info")
btn_visualizar = widgets.Button(description="Visualizar", layout=widgets.Layout(margin="10px 0px 15px 20px"), button_style="info")
btn_borrar = widgets.Button(description="Borrar", layout=widgets.Layout(margin="10px 0px 15px 20px"), button_style="info")
btn_cero = widgets.Button(description="Poner a cero", layout=widgets.Layout(margin="10px 0px 15px 20px"), button_style="success")

out_error = widgets.Output(layout=widgets.Layout(width="100%", margin="20px 0px 0px 0px"))
out2 = widgets.Output(layout=widgets.Layout(width="100%", margin="20px 0px 0px 0px"))
out3 = widgets.Output(layout=widgets.Layout(width="80%", margin="20px 0px 0px 0px"))

for n in nombres:
    intro_pesos_d[n] = widgets.BoundedFloatText(value=np.round(float(100/len(nombres)), 2),
                                             min=0,
                                             max=100,
                                             step=0.01, 
                                             readout_format='.2f',
                                             layout=widgets.Layout(position="center", margin='0px 0px 0px 10px'), 
                                             style=style)

boxes_intro_pesos = [widgets.HBox([widgets.Label(n), intro_pesos_d[n]], layout=widgets.Layout(justify_content="center")) for n in nombres]


    
def btn_anadir_onclick(x):
    tw = []
        
    for i in range(len(intro_pesos_d)):
        tw.append(list(intro_pesos_d.values())[i].value/100)
        
    tr = rent_cartera(tw, rentabilidades)
    tv = vol_cartera(tw, mat_covarianzas)

    if [tr, tv] not in extras:
        extras.append([tr, tv])
        info_extras.append(tw)
        
    with out2:
        out2.clear_output()
        draw()
        
        for i_e, e in enumerate(extras):
                plt.scatter(e[1], e[0], marker="o", s=80, c="red", edgecolors='black')
                plt.gca().annotate(i_e+1, (e[1]+0.0022, e[0]+0.0022), bbox=dict(boxstyle="round", fc="w"))
                
        plt.show()
    
    print_info()
    
    
def btn_vis_onclick(x):
    tw = []
    
    for i in range(len(intro_pesos_d)):
        tw.append(list(intro_pesos_d.values())[i].value/100)    

    with out2:
        out2.clear_output()
        draw()
        
        if len(extras) != 0:    
            for e in extras:
                plt.scatter(e[1], e[0], s=80, marker="o", c="red", edgecolors='black')
        
        plt.scatter(vol_cartera(tw, mat_covarianzas), rent_cartera(tw, rentabilidades), s=80, c="red", marker="o", edgecolors='black')
        plt.show()
        
    print_info()
     
        
def btn_bor_onclick(x):
    global extras
    global info_extras 
    
    extras = []
    info_extras = []
    
    with out2:
        out2.clear_output()
        draw()    
        plt.show()
    
    print_info()

       
def check_error(**displays):
    summ = 0
    
    for d in range(len(intro_pesos_d)):
        summ += list(intro_pesos_d.values())[d].value
    
    with out_error:
        out_error.clear_output(wait=True)
        display("Los pesos suman: "+str(np.round(summ, 2)))


def btn_cero_onclick(x):
    for n in nombres:
        intro_pesos_d[n].value = 0
    
    check_error()
        

    
# Callbacks
btn_anadir.on_click(btn_anadir_onclick)
btn_visualizar.on_click(btn_vis_onclick)
btn_borrar.on_click(btn_bor_onclick)
btn_cero.on_click(btn_cero_onclick)

out_error = widgets.interactive_output(check_error, intro_pesos_d)
out_error.layout.margin = "9px 0px 0px 30px"

# Layout
display(widgets.VBox([
    widgets.HBox([
        btn_anadir,
        btn_visualizar, 
        btn_borrar,
        out_error,
        btn_cero], layout=widgets.Layout(justify_content='space-around', align_items='stretch', width='auto')), 
    widgets.VBox(boxes_intro_pesos), 
    out2,
    out3]))

with out_error:
    display("Los pesos suman: %s" % str(np.round(len(nombres)*(np.round(100/len(nombres), 2)), 2)))